In [3]:
library(ggplot2)
library(plyr)
library(tidyverse)
library(GridLMM)
library(snpStats)
library(qq)
library(lme4qtl)
library(sjstats)
library(wesanderson)

traits <- c('movement_count','mean_movement_length','speed_in_movement_mean','dist_travelled')

f1s_df_ts <- read.csv("f1s_df_ts.csv")
parents_df_ts <- read.csv("parents_df_ts.csv")

f1s_df_36 <- read.csv("f1s_df_36.csv")
parents_df_36 <- read.csv("parents_df_36.csv")

grm <- as.matrix(read.table("female_parents.biSNP.sing.HW.gatk.GQfilter.MAF05.w100s1r08.rel"))
colnames(grm) <- rownames(grm) <- read.table("female_parents.biSNP.sing.HW.gatk.GQfilter.MAF05.w100s1r08.rel.id")[,2]

f1_grm <- grm[grep("F1",colnames(grm)),grep("F1",colnames(grm))]
p_grm <- grm[-grep("F1",colnames(grm)),-grep("F1",colnames(grm))]

f1s <- grep("_S",row.names(grm))
parental_grm <- as.matrix(grm)

family <- sapply(row.names(parental_grm),FUN=function(x){strsplit(x,"_")[[1]][4]})
row.names(parental_grm) <- colnames(parental_grm) <- family

In [2]:
f1s_df_ts <- f1s_df_ts[f1s_df_ts$Family %in% colnames(parental_grm),]


#Filtering as needed
combined_filtered_df <- f1s_df_ts
combined_filtered_df = combined_filtered_df[which(combined_filtered_df$Empty == F),]
combined_filtered_df = combined_filtered_df[which(!is.na(combined_filtered_df$Treatment)),]
combined_filtered_df = combined_filtered_df[which(!is.na(combined_filtered_df$Batch)),]
# Account for framerate
combined_filtered_df$dist_travelled <- combined_filtered_df$dist_travelled/10

#Ensure the flies still move some -- no dead flies!
# combined_filtered_df = combined_filtered_df[which(combined_filtered_df$movement_count > 10),]
# combined_filtered_df = combined_filtered_df[which(combined_filtered_df$movement_count < 10000),]
# combined_filtered_df = combined_filtered_df[which(combined_filtered_df$mean_of_mean_in_movement_speed < 100),]
# combined_filtered_df = combined_filtered_df[combined_filtered_df$vel_par_in_movement_mean > 5 & combined_filtered_df$vel_par_in_movement_mean < 100,]
# combined_filtered_df <- combined_filtered_df[which(combined_filtered_df$speed_in_movement_var < 500),]
# combined_filtered_df <- combined_filtered_df[which(combined_filtered_df$mean_mov < 20000),]


f1s_df_ts <- combined_filtered_df

#Append meaningful individual ID
f1s_df_ts$indiv_id <- paste0(f1s_df_ts$Batch,"_",f1s_df_ts$Family,"_",f1s_df_ts$Well_orderAsTracked)

In [3]:
f1s_df_36 <- f1s_df_36[f1s_df_36$Family %in% colnames(parental_grm),]


#Filtering as needed
combined_filtered_df <- f1s_df_36
combined_filtered_df = combined_filtered_df[which(combined_filtered_df$Empty == F),]
combined_filtered_df = combined_filtered_df[which(!is.na(combined_filtered_df$Treatment)),]
combined_filtered_df = combined_filtered_df[which(!is.na(combined_filtered_df$Batch)),]
# Account for framerate
combined_filtered_df$dist_travelled <- combined_filtered_df$dist_travelled/10

#Ensure the flies still move some -- no dead flies!
# combined_filtered_df = combined_filtered_df[which(combined_filtered_df$movement_count > 10),]
# combined_filtered_df = combined_filtered_df[which(combined_filtered_df$movement_count < 10000),]
# combined_filtered_df = combined_filtered_df[which(combined_filtered_df$mean_of_mean_in_movement_speed < 100),]
# combined_filtered_df = combined_filtered_df[combined_filtered_df$vel_par_in_movement_mean > 5 & combined_filtered_df$vel_par_in_movement_mean < 100,]
# combined_filtered_df <- combined_filtered_df[which(combined_filtered_df$speed_in_movement_var < 500),]
# combined_filtered_df <- combined_filtered_df[which(combined_filtered_df$mean_mov < 20000),]


f1s_df_36 <- combined_filtered_df

#Append meaningful individual ID
f1s_df_36$indiv_id <- paste0(f1s_df_36$Batch,"_",f1s_df_36$Family,"_",f1s_df_36$Well_orderAsTracked)

# Generate the individual level GRM
Here we build a matrix for each family

In [13]:
#We will need a list of matrices to create our block diag matrix later!
matrices = list()
names <- c()

#We also need to sort everything so that we can use 

nFams <- length(unique(f1s_df_36$Family))
nGenotypes <- nrow(parental_grm)

In [14]:
sorting_vec <- c()
for (family in unique(f1s_df_36$Family)) {
    n_indiv <- nrow(f1s_df_36[f1s_df_36$Family == toString(family), ])
    names <- append(names,rep(as.integer(family),n_indiv))
    sorting_vec <- append(sorting_vec,which(f1s_df_36$Family == toString(family)))
    # Defines within family relatedness
    matrices[[toString(family)]] = matrix(rep(.75, n_indiv ** 2), nrow = n_indiv)
    diag(matrices[[toString(family)]]) <- rep(1,n_indiv)
}

In [15]:
# Sort according to the ordering of our blocks
f1s_df_36 <- f1s_df_36[sorting_vec,]
# Expand parental grm according to number of individuals for each family
modified_grm <- matrix(nrow = length(names),ncol=length(names))
row.names(modified_grm) <- colnames(modified_grm) <- names
mgrm_cnames = colnames(modified_grm)
mgrm_rnames = rownames(modified_grm)

In [16]:
for (i in 1:nFams) {
  name = colnames(parental_grm)[i]
  for (j in i:nFams) {
    name_2= colnames(parental_grm)[j] 
    modified_grm[which(mgrm_rnames == name), which(mgrm_cnames == name_2)] <-
      modified_grm[which(mgrm_rnames == name_2), which(mgrm_cnames == name)] <-
      0.5#  + parental_grm[toString(name), toString(name_2)]/2 
      # Above defines the off diagonals! Probably the place to start for tweaking things.
  }
}

In [17]:
# Build diagonal matrix and fill values in our GRM
block_diag_mat <- as.matrix(bdiag(matrices))
modified_grm[which(block_diag_mat != 0)] = block_diag_mat[which(block_diag_mat != 0)]
rownames(modified_grm) <- colnames(modified_grm) <- names

In [18]:
f1s_df_36 <- f1s_df_36[sorting_vec,]
modified_grm_id_tagged <- modified_grm
colnames(modified_grm_id_tagged) <- rownames(modified_grm_id_tagged) <- f1s_df_36$indiv_id

In [20]:
h2_estimates <- data.frame(trait=character(),
                 treatment=character(),
                 h2=numeric(), 
                 slicetime=numeric(),
                 se=numeric(),
                 p=numeric()) 
traits <- c('movement_count','mean_movement_length','speed_in_movement_mean','dist_travelled')

In [32]:
# for (trait in traits){
#     print(trait)
#     for (treatment in c("C","R","HS")){
#         print(treatment)
#         for (time in c(0,32)){
#             print(time)
#             slice <- f1s_df_ts[f1s_df_ts$Treatment == treatment & f1s_df_ts$slicetime == time,]#mapping_df[mapping_df$Treatment == treatment & mapping_df$slicetime == time,]
#             # This drops more data than 
#             slice <- slice[slice$indiv_id %in% rownames(modified_grm_id_tagged),]
#             slice <- slice[1:100,]
#             form = as.formula(paste0(trait, "~","1 + (1 | indiv_id)"))
#             full_mod <- relmatLmer(form,slice,relmat = list(indiv_id = modified_grm_id_tagged),calc.derivs = FALSE)
#             dummy <- slice %>% 
#             # generate 10 bootstrap replicates of dataset
#             bootstrap(3) %>% 
#             # run mixed effects regression on each bootstrap replicate
#             mutate(models = lapply(.$strap, function(x) {
#               relmatLmer(as.formula(paste(trait, '1 + (1|indiv_id)', sep = ' ~ ')), data = x,
#                          relmat = list(indiv_id = modified_grm_id_tagged),calc.derivs = FALSE)
#             })) %>% 
#             # compute ICC for each "bootstrapped" regression
#             mutate(icc = unlist(lapply(.$models, function(x){icc(x)$ICC_adjusted})));
#             se <- boot_se(dummy, icc)[2]
#             p <- boot_p(dummy, icc)[2]
#             print(se)
#             h2_estimates[nrow(h2_estimates) + 1,] <- data.frame(trait,treatment,lme4qtl::VarProp(full_mod)$prop[1],time,se,p)
#         }
#     }
# }